In [2]:
import pandas as pd
import math
import pickle
import json
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
#Load docs, queries and the inverted index
with open("./Output/preprocessed_docs","rb") as file:
    docs = pickle.load(file)

with open("./Output/preprocessed_queries","rb") as file:
    queries = pickle.load(file)

with open("./Output/inverted_index","r") as file:
    inverted_index = json.load(file)


In [4]:
def query_weights1(tf,idf,term):
    return (0.5 + (0.5*tf[term])/tf[max(tf)]) * idf

In [5]:
def document_weights2(tf,idf,term):
    return tf

In [6]:
def document_weights1(tf,idf,term):
    return tf*idf

In [8]:
def vector_space(query,docs, query_weight_func, doc_weight_func):

        tf={}
        
        for term in set(query):#Get query term frequency
                tf[term] = query.count(term)

        #Calculate query and doc weights
        query_weights = []
        document_weights = {}

        for doc in docs:
                document_weights[doc[0]] = []
        
        for term in set(query):

                if term in inverted_index and len(inverted_index[term]):
                        
                        idf = math.log(len(docs)/len(inverted_index[term]))

                        query_weights.append(query_weight_func(tf,idf,term))

                        for doc in docs:

                                try:
                                        tf_doc = inverted_index[term][doc[0]]
                                except:
                                        tf_doc = 0

                                w = doc_weight_func(tf_doc,idf,term)
                                document_weights[doc[0]].append(w)   

       
        #Calcualte cosine similarity for each doc
        sim = {}
        for doc in document_weights:
                sim[doc] = cosine_similarity([query_weights],[document_weights[doc]])

        return sorted(sim.items(), key=lambda x:x[1])[-500:][::-1]

In [9]:

results = []

# Iterate over queries
for query in queries:
    # Call the vector_space function and get the result
    result = vector_space(query, docs, query_weights1, document_weights1)
    
    # Extract the second element of each tuple in the result and convert it to a list
    elements = [int(item[0]) for item in result]
    
    # Add the iteration number and the list of second elements to the result_dict

    results.append(elements)

In [10]:
with open("./Output/vsm_results_1", "wb") as file:
    pickle.dump(results, file)

In [11]:
results = []

# Iterate over queries
for query in queries:
    # Call the vector_space function and get the result
    result = vector_space(query, docs, query_weights1, document_weights2)
    
    # Extract the second element of each tuple in the result and convert it to a list
    elements = [int(item[0]) for item in result]
    
    # Add the iteration number and the list of second elements to the result_dict

    results.append(elements)

In [12]:
with open("./Output/vsm_results_2", "wb") as file:
    pickle.dump(results, file)